In [6]:
import autograd.numpy as np

import pymanopt
from pymanopt.manifolds import Elliptope
from pymanopt.optimizers import ConjugateGradient

SUPPORTED_BACKENDS = ("autograd")


def create_cost(manifold, epsilon, backend):
    @pymanopt.function.autograd(manifold)
    def cost(X):
        Y = X @ X.T
        # Shift the exponentials by the maximum value to reduce numerical
        # trouble due to possible overflows.
        s = np.triu(Y, 1).max()
        expY = np.exp((Y - s) / epsilon)
        # Zero out the diagonal
        expY -= np.diag(np.diag(expY))
        u = np.triu(expY, 1).sum()
        return s + epsilon * np.log(u)

    return cost


backend=SUPPORTED_BACKENDS[0]
quiet=False
dimension = 3  # Dimension of the embedding space, i.e. R^k
num_points = 24  # Points on the sphere
# This value should be as close to 0 as affordable. If it is too close to
# zero, optimization first becomes much slower, than simply doesn't work
# anymore because of floating point overflow errors (NaN's and Inf's start
# to appear). If it is too large, then log-sum-exp is a poor approximation
# of the max function, and the spread will be less uniform. An okay value
# seems to be 0.01 or 0.001 for example. Note that a better strategy than
# using a small epsilon straightaway is to reduce epsilon bit by bit and to
# warm-start subsequent optimization in that way. Trustregions will be more
# appropriate for these fine tunings.
epsilon = 0.005

manifold = Elliptope(num_points, dimension)
cost = create_cost(manifold, epsilon, backend)
problem = pymanopt.Problem(manifold, cost)
if quiet:
    problem.verbosity = 0

solver = ConjugateGradient()
Yopt = solver.run(problem)

Xopt = Yopt.point @ Yopt.point.T
maxdot = np.triu(Xopt, 1).max()
print("Maximum angle between any two points:", maxdot)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +9.9178369034971270e-01    1.61185762e-01    
   2         +9.7885958740670875e-01    1.75653999e-01    
   3         +9.7385822173786696e-01    1.91361628e-01    
   4         +9.5912868724741351e-01    1.74575485e-01    
   5         +9.5440175470635491e-01    2.27485040e-01    
   6         +9.4656085608850593e-01    1.90120050e-01    
   7         +9.3993692209258073e-01    1.62256747e-01    
   8         +9.3561791819940698e-01    1.45995498e-01    
   9         +9.3102287277283002e-01    1.27462475e-01    
  10         +9.2542464453778450e-01    1.92477901e-01    
  11         +9.1943217698788049e-01    1.76790949e-01    
  12         +9.1682485946536607e-01    1.62103172e-01    
  13         +9.1427819479671890e-01    1.20126738e-01    
  14         +9.1251634653280145e-01    1.10359123e-01    
  15         +9.1056268151130793e-01    1.

In [7]:
import autograd.numpy as anp
import pymanopt
import pymanopt.manifolds
import pymanopt.optimizers

anp.random.seed(42)

dim = 3
manifold = pymanopt.manifolds.Sphere(dim)

matrix = anp.random.normal(size=(dim, dim))
matrix = 0.5 * (matrix + matrix.T)

@pymanopt.function.autograd(manifold)
def cost(point):
    return -point @ matrix @ point

problem = pymanopt.Problem(manifold, cost)

optimizer = pymanopt.optimizers.SteepestDescent()
result = optimizer.run(problem)

eigenvalues, eigenvectors = anp.linalg.eig(matrix)
dominant_eigenvector = eigenvectors[:, eigenvalues.argmax()]

print("Dominant eigenvector:", dominant_eigenvector)
print("Pymanopt solution:", result.point)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +1.1041943339110254e+00    5.65626470e-01    
   2         +5.2849633289004561e-01    8.90742722e-01    
   3         -8.0741058657312559e-01    2.23937710e+00    
   4         -1.2667369971251594e+00    1.59671326e+00    
   5         -1.4100298597091836e+00    1.11228845e+00    
   6         -1.5219408277812503e+00    2.45507203e-01    
   7         -1.5269956262562046e+00    6.81712914e-02    
   8         -1.5273114803528711e+00    3.40941735e-02    
   9         -1.5273905588875485e+00    1.70222768e-02    
  10         -1.5274100956128573e+00    8.61140952e-03    
  11         -1.5274154319869830e+00    3.90706914e-03    
  12         -1.5274156215853552e+00    3.62943720e-03    
  13         -1.5274162595152965e+00    2.47643447e-03    
  14         -1.5274168030609059e+00    3.66398563e-04    
  15         -1.5274168133149573e+00    1.